# Quantile Regression for Drug Discovery with LightGBM Regressor

## Introduction of LightGBM
[LightGBM](https://github.com/Microsoft/LightGBM) is an open-source, distributed, high-performance gradient boosting framework with following advantages: 
-   Composability: LightGBM models can be incorporated into existing
    SparkML Pipelines, and used for batch, streaming, and serving
    workloads.
-   Performance: LightGBM on Spark is 10-30% faster than SparkML on
    the Higgs dataset, and achieves a 15% increase in AUC.  [Parallel
    experiments](https://github.com/Microsoft/LightGBM/blob/master/docs/Experiments.rst#parallel-experiment)
    have verified that LightGBM can achieve a linear speed-up by using
    multiple machines for training in specific settings.
-   Functionality: LightGBM offers a wide array of [tunable
    parameters](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.rst),
    that one can use to customize their decision tree system. LightGBM on
    Spark also supports new types of problems such as quantile regression.
-   Cross platform：LightGBM on Spark is available on Spark (Scala) and PySpark (Python).

<img src="https://mmlspark.blob.core.windows.net/graphics/Documentation/drug.png" width="800" style="float: center;"/>

In this example, we use LightGBM quantile regressor on the Triazines dataset.

## Read dataset

In [1]:
triazines = spark.read.format("libsvm").load("wasbs://publicwasb@mmlspark.blob.core.windows.net/triazines.scale.svmlight")

StatementMeta(SampleSpark, 44, 1, Finished, Available)

## Exploratory data

In [2]:
# print some basic info
print("records read: " + str(triazines.count()))
print("Schema: ")
triazines.printSchema()
triazines.limit(10).toPandas()

StatementMeta(SampleSpark, 44, 2, Finished, Available)

records read: 105
Schema: 
root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)

   label                                           features
0  0.809  (-0.6, -0.3325, -0.3325, -1.0, -1.0, -1.0, -1....
1  0.602  (-0.6, 0.0, 0.0, -1.0, -0.3325, -1.0, -1.0, 0....
2  0.442  (-0.6, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
3  0.718  (-0.6, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
4  0.697  (-0.6, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
5  0.757  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....
6  0.900  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....
7  0.564  (-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
8  0.772  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....
9  0.801  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....
/opt/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:2110: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported typ

## Generation of testing and training data sets

Simple split, 85% for training and 15% for testing the model. Playing with this ratio may result in different models.

In [3]:
train, test = triazines.randomSplit([0.85, 0.15], seed=1)

StatementMeta(SampleSpark, 44, 3, Finished, Available)

## Train the model
Train the quantile regressor on the training data.

In [4]:
from mmlspark.lightgbm import LightGBMRegressor
model = LightGBMRegressor(objective='quantile',
                          alpha=0.2,
                          learningRate=0.3,
                          numLeaves=31).fit(train)

StatementMeta(SampleSpark, 44, 4, Finished, Available)

In [5]:
# Save and load LightGBM to a file using the LightGBM native representation

from mmlspark.lightgbm import LightGBMRegressionModel
model.saveNativeModel("/mymodel")
model = LightGBMRegressionModel.loadNativeModelFromFile("/mymodel")

StatementMeta(SampleSpark, 44, 5, Finished, Available)

In [6]:
# View the feature importances of the trained model

print(model.getFeatureImportances())

StatementMeta(SampleSpark, 44, 6, Finished, Available)

[18.0, 4.0, 8.0, 0.0, 16.0, 16.0, 0.0, 3.0, 2.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 27.0, 27.0, 18.0, 28.0, 28.0, 0.0, 10.0, 0.0, 4.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 0.0]

## Model performance evaluation

After training the model, we evaluate the performance of the model using the test set.

In [7]:
scoredData = model.transform(test)
scoredData.limit(10).toPandas()

StatementMeta(SampleSpark, 44, 7, Finished, Available)

   label                                           features  prediction
0  0.258  (-0.2, 0.3325, -0.6675, -1.0, 0.3325, 0.0, -1....    0.414115
1  0.427  (-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....    0.539532
2  0.550  (-0.6, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....    0.537624
3  0.614  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....    0.640256
4  0.631  (-0.6, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....    0.422801
5  0.637  (-0.2, 0.0, 0.0, -1.0, 0.3325, 0.0, -1.0, 0.0,...    0.521593
6  0.641  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....    0.585361
7  0.678  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....    0.585361
8  0.788  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....    0.726604
9  0.801  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....    0.634850

In [8]:
# Compute metrics using ComputeModelStatistics

from mmlspark.train import ComputeModelStatistics
metrics = ComputeModelStatistics(evaluationMetric='regression',
                                 labelCol='label',
                                 scoresCol='prediction') \
            .transform(scoredData)
metrics.toPandas()

StatementMeta(SampleSpark, 44, 8, Finished, Available)

   mean_squared_error  root_mean_squared_error       R^2  mean_absolute_error
0            0.014862                  0.12191  0.495869             0.107673

## Clean up resources
To ensure the Spark instance is shut down, end any connected sessions(notebooks). The pool shuts down when the **idle time** specified in the Apache Spark pool is reached. You can also select **stop session** from the status bar at the upper right of the notebook.

![stopsession](https://adsnotebookrelease.blob.core.windows.net/adsnotebookrelease/adsnotebook/image/stopsession.png)

## Next steps

* [Check out Synapse sample notebooks](https://github.com/Azure-Samples/Synapse/tree/main/MachineLearning) 
* [MMLSpark GitHub Repo](https://github.com/Azure/mmlspark)